In [4]:
import os
import random as _random
import numpy

In [26]:
RND = 1
N_HOLDOUT = 2500
N_VAL = 2500
N_TRAIN = 500000
OUT_DIR = 'out_500k'

In [6]:
_random.seed(RND)
numpy.random.seed(RND)
if not os.path.isdir(OUT_DIR): os.makedirs(OUT_DIR)

In [7]:
%run '../data-generator.ipynb'

In [ ]:
dg = DataGenerator('/d2/caches/tf-speech/train/audio')

In [25]:
# get some stats on input data
counts = {k: len(v) for k, v in dg.input_files.items()}
print(counts)
del counts['unknown']
print('total samples for known words:', sum(counts.values()))
print('avg per known label:', sum(counts.values()) / len(counts))

{'down': 2359, 'yes': 2377, 'left': 2353, 'right': 2367, 'go': 2372, 'stop': 2380, 'unknown': 41039, 'up': 2375, 'on': 2367, 'off': 2357, 'no': 2375}
total samples for known words: 23682
avg per known label: 2368.2


In [21]:
def _gen():

    # generate, save
    dg.val_files = {}
    val_X, val_Y = dg.generate_val_set(n=N_VAL_SAMPLES)
    np.save('%s/val_files.npy' % OUT_DIR, dg.val_files)
    np.save('%s/val_X.npy' % OUT_DIR, val_X)
    np.save('%s/val_Y.npy' % OUT_DIR, val_Y)

    dg.generate_train_set(
        n_total=N_TRAIN_SAMPLES,
        n_per_job=1000,
        n_pools=16,
        X_file='%s/train_X.mem' % OUT_DIR,
        Y_file='%s/train_Y.mem' % OUT_DIR,
        tmp_dir='%s/train_tmp' % OUT_DIR)

SyntaxError: invalid syntax (<ipython-input-21-b502e7355696>, line 1)

In [5]:
%time _gen()

CPU times: user 3min 29s, sys: 9min 18s, total: 12min 47s
Wall time: 11h 29min 48s
